In [55]:
import pandas as pd
import numpy as np # linear algebra
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.metrics import confusion_matrix,accuracy_score
from subprocess import check_output
print(check_output(["ls", "/Users/lexis/FIA/yelp"]).decode("utf8"));



Dataset_Challenge_Dataset_Agreement.pdf
output.csv
yelp_business.csv
yelp_business_attributes.csv
yelp_business_hours.csv
yelp_checkin.csv
yelp_review.csv
yelp_tip.csv
yelp_user.csv



In [32]:
yelp_business = pd.read_csv('/Users/lexis/FIA/yelp/yelp_business.csv')

In [33]:
review = pd.read_csv('/Users/lexis/FIA/yelp/yelp_review.csv')
#checkin = pd.read_csv('../input/yelp_checkin.csv')

review_busines = review.groupby(by='business_id')

review_businesid = pd.DataFrame()
review_businesid['Mean'] = review_busines['stars'].mean()
review_businesid['Median'] = review_busines['stars'].median()
review_businesid['NumberOfReviews'] = review_busines['stars'].count()

In [34]:
checkin = pd.read_csv("/Users/lexis/FIA/yelp/yelp_checkin.csv")

checkin_bus = checkin.groupby(by='business_id')

checkin_busid = pd.DataFrame()
checkin_busid['TotalCheckins'] = checkin_bus['checkins'].sum()

checkin_busid.reset_index(inplace=True)
review_businesid.reset_index(inplace=True)


review_businesid = pd.merge(left=review_businesid,right=checkin_busid,on='business_id', how='left')

In [35]:
yelp_business = pd.merge(left=yelp_business,right=review_businesid,on='business_id',how='left')

In [36]:
yelp_business.columns

Index([u'business_id', u'name', u'neighborhood', u'address', u'city', u'state',
       u'postal_code', u'latitude', u'longitude', u'stars', u'review_count',
       u'is_open', u'categories', u'Mean', u'Median', u'NumberOfReviews',
       u'TotalCheckins'],
      dtype='object')

In [67]:
cols1 = ['stars',
 'review_count',
        'Mean', 'Median', 'NumberOfReviews','TotalCheckins', 'latitude', 'longitude']

X = yelp_business[cols1]
y = yelp_business['is_open']

X.fillna(0.0,inplace=True)

In [73]:
yelp_business[['stars',
 'review_count',
        'Mean', 'Median', 'NumberOfReviews','TotalCheckins', 'latitude', 'longitude', 'is_open']].to_csv("/Users/lexis/FIA/yelp/output.csv", index = False)

In [38]:
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size = 0.3, random_state = 42)

In [39]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(train_X, train_y)
X_res = pd.DataFrame(X_res,columns = train_X.columns.tolist())
y_res = pd.DataFrame(y_res)
test_X = pd.DataFrame(test_X)
test_y = pd.DataFrame(test_y)

In [40]:
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, seed_val=2017, num_rounds=400):
    param = {}
    param['objective'] = 'multi:softmax'
    param['num_class'] = 2
    param['eta'] = 0.12
    param['max_depth'] = 5
    param['silent'] = 1
    param['eval_metric'] = 'merror'
    param['min_child_weight'] = 1
    param['subsample'] = 0.5
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return model,pred_test_y

In [43]:
test_y_pred= pd.DataFrame()
test_y_pred['id'] = test_y.index
test_y_pred['is_open'] = np.zeros_like(test_y.index)

In [44]:
X_res = X_res.values
y_res = y_res.values
test_X = test_X.values

In [49]:
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=42)

for i, (train_index, test_index) in enumerate(skf.split(X_res, y_res)):
    X_train, X_valid = X_res[train_index], X_res[test_index]
    y_train, y_valid = y_res[train_index], y_res[test_index]
    
    model1,y = runXGB( X_train,y_train,X_valid,y_valid)
    test_pred = model1.predict(xgb.DMatrix(test_X))
    test_y_pred['is_open'] += test_pred/kfold

[0]	train-merror:0.273562	test-merror:0.42326
Multiple eval metrics have been passed: 'test-merror' will be used for early stopping.

Will train until test-merror hasn't improved in 20 rounds.
[1]	train-merror:0.24566	test-merror:0.433796
[2]	train-merror:0.239863	test-merror:0.431752
[3]	train-merror:0.231797	test-merror:0.430827
[4]	train-merror:0.23274	test-merror:0.431144
[5]	train-merror:0.223233	test-merror:0.427299
[6]	train-merror:0.211128	test-merror:0.433017
[7]	train-merror:0.204844	test-merror:0.438613
[8]	train-merror:0.201578	test-merror:0.438516
[9]	train-merror:0.200623	test-merror:0.437616
[10]	train-merror:0.199644	test-merror:0.439075
[11]	train-merror:0.192107	test-merror:0.443917
[12]	train-merror:0.191894	test-merror:0.441265
[13]	train-merror:0.185136	test-merror:0.448516
[14]	train-merror:0.187028	test-merror:0.444063
[15]	train-merror:0.187289	test-merror:0.44708
[16]	train-merror:0.182624	test-merror:0.448832
[17]	train-merror:0.183512	test-merror:0.453674
[18

[143]	train-merror:0.187879	test-merror:0.122214
[144]	train-merror:0.18763	test-merror:0.122019
[145]	train-merror:0.187295	test-merror:0.121752
[146]	train-merror:0.187259	test-merror:0.121703
[147]	train-merror:0.186924	test-merror:0.121363
[148]	train-merror:0.186912	test-merror:0.121509
[149]	train-merror:0.186651	test-merror:0.121484
[150]	train-merror:0.186705	test-merror:0.121217
[151]	train-merror:0.186456	test-merror:0.12073
[152]	train-merror:0.18648	test-merror:0.120657
[153]	train-merror:0.186359	test-merror:0.1209
[154]	train-merror:0.185982	test-merror:0.120365
[155]	train-merror:0.185744	test-merror:0.120292
[156]	train-merror:0.18569	test-merror:0.120146
[157]	train-merror:0.185477	test-merror:0.119757
[158]	train-merror:0.185106	test-merror:0.119367
[159]	train-merror:0.185124	test-merror:0.119562
[160]	train-merror:0.184862	test-merror:0.119027
[161]	train-merror:0.18468	test-merror:0.118929
[162]	train-merror:0.184491	test-merror:0.118905
[163]	train-merror:0.184516

[312]	train-merror:0.166882	test-merror:0.099538
[313]	train-merror:0.166827	test-merror:0.099416
[314]	train-merror:0.166717	test-merror:0.099027
[315]	train-merror:0.166468	test-merror:0.098686
[316]	train-merror:0.166498	test-merror:0.09871
[317]	train-merror:0.166292	test-merror:0.09854
[318]	train-merror:0.166206	test-merror:0.098662
[319]	train-merror:0.166127	test-merror:0.098418
[320]	train-merror:0.166073	test-merror:0.098297
[321]	train-merror:0.166018	test-merror:0.098443
[322]	train-merror:0.166048	test-merror:0.098443
[323]	train-merror:0.165854	test-merror:0.098321
[324]	train-merror:0.165835	test-merror:0.098394
[325]	train-merror:0.165702	test-merror:0.098443
[326]	train-merror:0.165702	test-merror:0.09854
[327]	train-merror:0.165677	test-merror:0.098637
[328]	train-merror:0.16561	test-merror:0.098589
[329]	train-merror:0.165629	test-merror:0.098394
[330]	train-merror:0.165513	test-merror:0.098248
[331]	train-merror:0.165373	test-merror:0.098078
[332]	train-merror:0.165

[79]	train-merror:0.203062	test-merror:0.142506
[80]	train-merror:0.202874	test-merror:0.142652
[81]	train-merror:0.202892	test-merror:0.142871
[82]	train-merror:0.202764	test-merror:0.142311
[83]	train-merror:0.202569	test-merror:0.141898
[84]	train-merror:0.202314	test-merror:0.141825
[85]	train-merror:0.201882	test-merror:0.141168
[86]	train-merror:0.201773	test-merror:0.141265
[87]	train-merror:0.201128	test-merror:0.140268
[88]	train-merror:0.201037	test-merror:0.140073
[89]	train-merror:0.200818	test-merror:0.139562
[90]	train-merror:0.20055	test-merror:0.138832
[91]	train-merror:0.200319	test-merror:0.138491
[92]	train-merror:0.200404	test-merror:0.138564
[93]	train-merror:0.199844	test-merror:0.137591
[94]	train-merror:0.199564	test-merror:0.137397
[95]	train-merror:0.199102	test-merror:0.136959
[96]	train-merror:0.199315	test-merror:0.137007
[97]	train-merror:0.198993	test-merror:0.13708
[98]	train-merror:0.198841	test-merror:0.136642
[99]	train-merror:0.198609	test-merror:0.1

[248]	train-merror:0.173512	test-merror:0.105839
[249]	train-merror:0.173256	test-merror:0.105693
[250]	train-merror:0.173238	test-merror:0.105669
[251]	train-merror:0.173019	test-merror:0.10545
[252]	train-merror:0.173196	test-merror:0.105231
[253]	train-merror:0.173086	test-merror:0.105231
[254]	train-merror:0.172995	test-merror:0.105231
[255]	train-merror:0.172843	test-merror:0.105182
[256]	train-merror:0.172855	test-merror:0.105158
[257]	train-merror:0.172612	test-merror:0.104623
[258]	train-merror:0.172362	test-merror:0.104501
[259]	train-merror:0.172289	test-merror:0.104647
[260]	train-merror:0.172034	test-merror:0.104599
[261]	train-merror:0.17207	test-merror:0.104453
[262]	train-merror:0.171973	test-merror:0.104209
[263]	train-merror:0.171882	test-merror:0.103917
[264]	train-merror:0.171851	test-merror:0.103771
[265]	train-merror:0.171711	test-merror:0.103796
[266]	train-merror:0.171717	test-merror:0.103674
[267]	train-merror:0.171699	test-merror:0.103528
[268]	train-merror:0.1

[14]	train-merror:0.257807	test-merror:0.216277
[15]	train-merror:0.255843	test-merror:0.213358
[16]	train-merror:0.251402	test-merror:0.207518
[17]	train-merror:0.252564	test-merror:0.209854
[18]	train-merror:0.25257	test-merror:0.209465
[19]	train-merror:0.250106	test-merror:0.206861
[20]	train-merror:0.248373	test-merror:0.205523
[21]	train-merror:0.247181	test-merror:0.203698
[22]	train-merror:0.244869	test-merror:0.200438
[23]	train-merror:0.243878	test-merror:0.199221
[24]	train-merror:0.240818	test-merror:0.195012
[25]	train-merror:0.238513	test-merror:0.192165
[26]	train-merror:0.23552	test-merror:0.187616
[27]	train-merror:0.235289	test-merror:0.18635
[28]	train-merror:0.234115	test-merror:0.184526
[29]	train-merror:0.232071	test-merror:0.181363
[30]	train-merror:0.230641	test-merror:0.179805
[31]	train-merror:0.23055	test-merror:0.179246
[32]	train-merror:0.229729	test-merror:0.178394
[33]	train-merror:0.229176	test-merror:0.177908
[34]	train-merror:0.227156	test-merror:0.175

[184]	train-merror:0.180799	test-merror:0.115888
[185]	train-merror:0.180647	test-merror:0.115742
[186]	train-merror:0.180367	test-merror:0.115158
[187]	train-merror:0.180288	test-merror:0.115158
[188]	train-merror:0.180124	test-merror:0.115353
[189]	train-merror:0.179984	test-merror:0.115012
[190]	train-merror:0.179905	test-merror:0.115109
[191]	train-merror:0.179783	test-merror:0.115061
[192]	train-merror:0.179613	test-merror:0.114696
[193]	train-merror:0.179528	test-merror:0.114404
[194]	train-merror:0.179522	test-merror:0.114234
[195]	train-merror:0.17937	test-merror:0.114209
[196]	train-merror:0.179005	test-merror:0.113917
[197]	train-merror:0.178798	test-merror:0.113577
[198]	train-merror:0.178767	test-merror:0.113455
[199]	train-merror:0.178743	test-merror:0.11309
[200]	train-merror:0.178725	test-merror:0.11309
[201]	train-merror:0.178719	test-merror:0.112968
[202]	train-merror:0.178457	test-merror:0.112774
[203]	train-merror:0.178439	test-merror:0.112798
[204]	train-merror:0.17

[352]	train-merror:0.164649	test-merror:0.098005
[353]	train-merror:0.164576	test-merror:0.097956
[354]	train-merror:0.164521	test-merror:0.097859
[355]	train-merror:0.164327	test-merror:0.097494
[356]	train-merror:0.164333	test-merror:0.097518
[357]	train-merror:0.164302	test-merror:0.097494
[358]	train-merror:0.164059	test-merror:0.097372
[359]	train-merror:0.163974	test-merror:0.097299
[360]	train-merror:0.163889	test-merror:0.097348
[361]	train-merror:0.163828	test-merror:0.097251
[362]	train-merror:0.163798	test-merror:0.097105
[363]	train-merror:0.163664	test-merror:0.097129
[364]	train-merror:0.163548	test-merror:0.097032
[365]	train-merror:0.163585	test-merror:0.097324
[366]	train-merror:0.163506	test-merror:0.097178
[367]	train-merror:0.163427	test-merror:0.09708
[368]	train-merror:0.163281	test-merror:0.096861
[369]	train-merror:0.163092	test-merror:0.096764
[370]	train-merror:0.163092	test-merror:0.096715
[371]	train-merror:0.163056	test-merror:0.096594
[372]	train-merror:0.

[120]	train-merror:0.194094	test-merror:0.129471
[121]	train-merror:0.19385	test-merror:0.128984
[122]	train-merror:0.19351	test-merror:0.128838
[123]	train-merror:0.1934	test-merror:0.128473
[124]	train-merror:0.192731	test-merror:0.127305
[125]	train-merror:0.192324	test-merror:0.126965
[126]	train-merror:0.191466	test-merror:0.125821
[127]	train-merror:0.19146	test-merror:0.125481
[128]	train-merror:0.191247	test-merror:0.124921
[129]	train-merror:0.19101	test-merror:0.124897
[130]	train-merror:0.190535	test-merror:0.124142
[131]	train-merror:0.190468	test-merror:0.124215
[132]	train-merror:0.190383	test-merror:0.124094
[133]	train-merror:0.190122	test-merror:0.123875
[134]	train-merror:0.189994	test-merror:0.123631
[135]	train-merror:0.189818	test-merror:0.123437
[136]	train-merror:0.189696	test-merror:0.123364
[137]	train-merror:0.18969	test-merror:0.123631
[138]	train-merror:0.189367	test-merror:0.123339
[139]	train-merror:0.189136	test-merror:0.123315
[140]	train-merror:0.189106

[288]	train-merror:0.169714	test-merror:0.099372
[289]	train-merror:0.169745	test-merror:0.099251
[290]	train-merror:0.169617	test-merror:0.099007
[291]	train-merror:0.169428	test-merror:0.098959
[292]	train-merror:0.169446	test-merror:0.098959
[293]	train-merror:0.169465	test-merror:0.099056
[294]	train-merror:0.16938	test-merror:0.098934
[295]	train-merror:0.1691	test-merror:0.09891
[296]	train-merror:0.169002	test-merror:0.098618
[297]	train-merror:0.16899	test-merror:0.098667
[298]	train-merror:0.169148	test-merror:0.098861
[299]	train-merror:0.169191	test-merror:0.098691
[300]	train-merror:0.169094	test-merror:0.098837
[301]	train-merror:0.168887	test-merror:0.098642
[302]	train-merror:0.168826	test-merror:0.098813
[303]	train-merror:0.16882	test-merror:0.098886
[304]	train-merror:0.168704	test-merror:0.098886
[305]	train-merror:0.168546	test-merror:0.099056
[306]	train-merror:0.1684	test-merror:0.098837
[307]	train-merror:0.168297	test-merror:0.098934
[308]	train-merror:0.168157	

In [50]:
test_y_pred['is_open'] = np.round(test_y_pred['is_open']).apply(int)

In [51]:
accuracy_score(test_y,test_y_pred['is_open'])

0.8113459739168624

In [56]:
confusion_matrix(test_y,test_y_pred['is_open'])

array([[ 1732,  6686],
       [ 3194, 40759]])